In [ ]:
import numpy as np
from netCDF4 import Dataset

# Open the NetCDF file
file_path = 'data_25_degree.nc'
fhadj = Dataset(file_path, mode='r')

# Extract original data
lon_sic = fhadj.variables['lon'][:120]  # Longitude
lat_sic = fhadj.variables['lat'][:120]  # Latitude (original: -90 to -60)
SIC = fhadj.variables['SIC'][:]  # Sea Ice Concentration (shape: time × lat × lon)
time_SIC = fhadj.variables['time'][:]  # Time

# Close the file
fhadj.close()

# Reverse latitude and SIC data
lat_sic_reversed = lat_sic[::-1]  # Now goes from -60 to -90
SIC_reversed = SIC[:, ::-1, :]  # Reverse along latitude (axis=1)

SIC_reversed = np.where(SIC_reversed > 1, 0, SIC_reversed)  # Set negative values to zero
lon_sic_reversed = lon_sic[::-1] 



# Verify
print("Original latitude range:", lat_sic[0], "to", lat_sic[-1])
print("Reversed latitude range:", lat_sic_reversed[0], "to", lat_sic_reversed[-1])
print("SIC shape before reversal:", SIC.shape)
print("SIC shape after reversal:", SIC_reversed.shape)
print("Min SIC:", np.min(SIC_reversed))
print("Max SIC:", np.max(SIC_reversed))

In [ ]:
import numpy as np

# Assuming time_SIC is in YYYYMMDD format (e.g., 20190101)
time_SIC_dates = time_SIC.astype(int)  # Ensure it's integer if not already

# Define the start and end dates in YYYYMMDD format
start_date = 20191101
end_date = 20200301

# Find indices where time_SIC is within the desired range
mask = (time_SIC_dates >= start_date) & (time_SIC_dates < end_date)
time_indices = np.where(mask)[0]

print(f"Found {len(time_indices)} time steps between {start_date} and {end_date}")

# Extract the SIC data for the selected time range
selected_SIC = SIC_reversed[time_indices, -120:, :]  # Shape: (time, lat, lon)

# Verify the extracted data
print("\n--- Extracted Data Summary ---")
print(f"Shape of selected_SIC: {selected_SIC.shape}")
print(f"First date in selection: {time_SIC_dates[time_indices[0]]}")
print(f"Last date in selection: {time_SIC_dates[time_indices[-1]]}")
print(f"Min SIC in selection: {np.min(selected_SIC)}")
print(f"Max SIC in selection: {np.max(selected_SIC)}")

# Close the NetCDF file


# November 2019: 30 days → indices 0 to 29
nov_indices = list(range(0, 30))

# December 2019: 31 days → indices 30 to 60
dec_indices = list(range(30, 61))

# January 2020: 31 days → indices 61 to 91
jan_indices = list(range(61, 92))

# February 2020: 29 days (leap year) → indices 92 to 120
feb_indices = list(range(92, 121))


print(nov_indices)
print(dec_indices)
print(jan_indices)
print(feb_indices)


In [ ]:
import cv2

In [ ]:
import numpy as np

# Initialize an empty list to store daily differences
SIC_daily_diff_list = []

# Loop through selected_SIC and compute differences
for i in range(len(selected_SIC) - 1):  # Stop at second-to-last day
    daily_diff = selected_SIC[i+1] - selected_SIC[i]  # day[i+1] - day[i]
    SIC_daily_diff_list.append(625*daily_diff)

# Convert the list to a NumPy array (optional, but useful for further analysis)
SIC_daily_diff_array = np.array(SIC_daily_diff_list)

# Verify results
print("--- Daily Differences (Loop Method) ---")
print(f"Number of computed differences: {len(SIC_daily_diff_list)}")
print(f"Shape of first difference: {SIC_daily_diff_list[0].shape}")


In [ ]:
import numpy as np
from netCDF4 import Dataset

# Open the NetCDF files
filtered_file_1 = '2019_4_month.nc'
fhadj_1 = Dataset(filtered_file_1, mode='r')

#61 is staring of November

ice_sheets_1 = fhadj_1.variables['sd'][61:]
ice_sheets_1_lat = fhadj_1.variables['latitude'][-120:]
ice_sheets_1_lon_before_t = fhadj_1.variables['longitude'][:]

filtered_file_2 = '2020_jan_feb.nc'
fhadj_2 = Dataset(filtered_file_2, mode='r')
ice_sheets_2 = fhadj_2.variables['sd'][:]

# Close the NetCDF files
fhadj_1.close()
fhadj_2.close()

# Concatenate ice_sheets_1 and ice_sheets_2 along the time axis
combined_ice_sheets_before_t = np.concatenate((ice_sheets_1, ice_sheets_2), axis=0)

# Convert longitude from 0–360 to -180–180
ice_sheets_1_lon_after_180 = np.where(
    ice_sheets_1_lon_before_t > 180,
    ice_sheets_1_lon_before_t - 360,
    ice_sheets_1_lon_before_t
)

# Find where the longitude wraps and roll data to make it continuous
split_idx = np.argmax(ice_sheets_1_lon_after_180 < 0)
combined_ice_sheets = np.roll(combined_ice_sheets_before_t, -split_idx, axis=2)
ice_sheets_1_lon = np.roll(ice_sheets_1_lon_after_180, -split_idx)

# Compute daily differences
daily_diff = 625*np.diff(combined_ice_sheets, axis=0)

# Slice to keep only latitudes 0:120 (and all longitudes)
daily_diff_sliced = daily_diff[:, -120:, :]

print(f"Shape of sliced daily differences: {daily_diff_sliced.shape}")
print(f"Longitude range after conversion: {ice_sheets_1_lon.min()}, {ice_sheets_1_lon.max()}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'selected_SIC' and 'combined_ice_sheets' are already defined
# Select the first time step for demonstration (index 0)
sic_for_graph_building = SIC_daily_diff_array[:, :, 200:600]  # SIC data for the first time step
ice_sheets_for_graph_building = daily_diff_sliced[:, -120:, 200:600]

sic_for_graph_building_plot = selected_SIC[:, :, 200:600]  # SIC data for the first time step
ice_sheets_for_graph_building_plot = combined_ice_sheets[:, -120:, 200:600]






In [ ]:
print('lat_sic')

lat_sic_reversed = lat_sic_reversed[:,0]

print(lat_sic_reversed.shape)
print('lon_sic')
print(lat_sic_reversed[0])

lon_sic_reversed=lon_sic_reversed[0,200:600]

print(lon_sic_reversed.shape)

print(lon_sic_reversed[0])




print('lat_ice')
print(ice_sheets_1_lat.shape)
print(ice_sheets_1_lat[0])

ice_sheets_1_lon = ice_sheets_1_lon[200:600]
print('lon_ice')
print(ice_sheets_1_lon.shape)
print(ice_sheets_1_lon[0])






In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extract first time step data
sic_data = sic_for_graph_building_plot[0]            # shape: (lat, lon)
ice_data = ice_sheets_for_graph_building_plot[0]    # shape: (lat, lon)

# Latitude and longitude ranges (optional for extent)
lat_range = ice_sheets_1_lat                     # shape: (120,)
lon_range = ice_sheets_1_lon_after_180           # shape: (421,)

# Create side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot Sea Ice Concentration
cax1 = axes[0].imshow(sic_data, cmap='Blues', interpolation='none', alpha=0.9)
axes[0].set_title('Sea Ice Concentration')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
fig.colorbar(cax1, ax=axes[0], label='SIC')

# Plot Ice Sheets
cax2 = axes[1].imshow(ice_data, cmap='Reds', interpolation='none', alpha=0.9)
axes[1].set_title('Ice Sheets')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
fig.colorbar(cax2, ax=axes[1], label='Ice Thickness')

# Adjust layout and show
plt.tight_layout()
plt.show()


In [ ]:
from scipy.spatial import Delaunay
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from tqdm import tqdm
from scipy import stats
from collections import deque
from concurrent.futures import ThreadPoolExecutor
import warnings
from scipy.spatial import cKDTree
# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning)

import numpy as np
import networkx as nx
from scipy.spatial import cKDTree

from scipy.spatial import Delaunay
import numpy as np
import networkx as nx



thres_defs = [
    "median–q3 (SIC), median–q3 (ICE)",
    "median–q3 (SIC), q3–ub (ICE)",
    "median–q3 (SIC), > ub (ICE)",
    "q3–ub (SIC), median–q3 (ICE)",
    "q3–ub (SIC), q3–ub (ICE)",
    "q3–ub (SIC), > ub (ICE)",
    "> ub (SIC), median–q3 (ICE)",
    "> ub (SIC), q3–ub (ICE)",
    "> ub (SIC), > ub (ICE)"
]

thres_def = thres_defs[0]


def is_in_range(val, thresholds, range_type):
    if range_type == 'r1':
        return thresholds['median'] <= val <= thresholds['q3']
    elif range_type == 'r2':
        return thresholds['q3'] < val <= thresholds['ub']
    elif range_type == 'r3':
        return val > thresholds['ub']


def create_graph(sic_changes, ice_changes, sic_thresholds, ice_thresholds):
    try:
        ocean_points = np.argwhere(sic_changes >0)

        land_points = np.argwhere(ice_changes >0)


        # print(ocean_points.shape)
        # print(land_points.shape)
        # print('after removing cimmmn points')
        # Convert coordinate arrays to sets of tuples for comparison
        ocean_tuples = {tuple(point) for point in ocean_points}
        land_tuples = {tuple(point) for point in land_points}

        # Find overlapping coordinates
        common_points = ocean_tuples & land_tuples

        if common_points:
            #print(f"Warning: {len(common_points)} locations have both sea ice and ice sheets:")
            # for point in common_points:
            #     print(f"  - Row {point[0]}, Column {point[1]}")
                
            # Treat common points as sea ice
            # Remove common points from the sea ice set
            #ocean_tuples -= common_points
            land_tuples -= common_points
            

            ##print("Common points are now treated as sea ice, and removed from the land ice set.")

        # else:
        #     print("No overlapping locations found - sea ice and ice sheets are properly separated")

        # Now continue with your logic
            

        
        ocean_points = np.array(list(ocean_tuples))
        land_points = np.array(list(land_tuples))
    

        # print(ocean_points.shape)
        # print(land_points.shape)

        # print('next')

        
        if len(ocean_points) + len(land_points) < 4:
            #print("Not enough points for triangulation")
            return None


        all_points = np.vstack([ocean_points, land_points])
        labels = ['O'] * len(ocean_points) + ['L'] * len(land_points)

        tri = Delaunay(all_points)

        point_labels = {
            tuple(p): f"{label}({p[0]},{p[1]})"
            for p, label in zip(all_points, labels)
        }

        G = nx.Graph()
        G.graph['ocean_points'] = ocean_points
        G.graph['land_points'] = land_points

        #Add all nodes with correct change values
        for point, label in zip(all_points, labels):
            row, col = point
            node_name = point_labels[tuple(point)]
            
            if label == 'O':
                change_val = sic_changes[row, col]
                G.add_node(node_name, label='O', sic_data=change_val)
            else:
                change_val = ice_changes[row, col]
                G.add_node(node_name, label='L', ice_data=change_val)


       

        for simplex in tri.simplices:
            for i in range(3):
                for j in range(i + 1, 3):
                    p1 = tuple(all_points[simplex[i]])
                    p2 = tuple(all_points[simplex[j]])
                    node1, node2 = point_labels[p1], point_labels[p2]

                    # Compute Euclidean distance
                    dist = np.linalg.norm(np.array(p1) - np.array(p2))
                    if dist > 11:
                        continue  # skip if distance is greater than 8

                    # Get changes for the nodes
                    def get_change(point, label):
                        row, col = point
                        return sic_changes[row, col] if label == 'O' else ice_changes[row, col]

                    label1, label2 = node1[0], node2[0]
                    change1 = get_change(p1, label1)
                    change2 = get_change(p2, label2)

                    # Apply thresholds
                    # Activate only this block (SIC r1, ICE r2)
                    if label1 == 'O' and not is_in_range(change1, sic_thresholds, 'r1'):
                        continue
                    if label1 == 'L' and not is_in_range(change1, ice_thresholds, 'r1'):
                        continue
                    if label2 == 'O' and not is_in_range(change2, sic_thresholds, 'r1'):
                        continue
                    if label2 == 'L' and not is_in_range(change2, ice_thresholds, 'r1'):
                        continue





                    weight = 1 if (change1 * change2 > 0) else -1

                    if (label1 == 'O' and label2 == 'L') or (label1 == 'L' and label2 == 'O'):
                        G.add_edge(
                            node1, node2,
                            weight=weight,
                            distance=dist,
                            sic_data=change1 if label1 == 'O' else change2,
                            ice_data=change2 if label2 == 'L' else change1
                        )
                    else:
                        G.add_edge(
                            node1, node2,
                            weight=weight,
                            distance=dist
                        )

        return G

    except Exception as e:
        print(f"Delaunay-based graph creation failed: {str(e)}")
        return None


import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from scipy.spatial import cKDTree

def find_positive_paths(all_G, sic_thresholds, ice_thresholds, time_range, max_length, M=500, n_workers=4):
    """Complete pipeline for positive correlation analysis"""
    # 1. Filter valid graphs
    valid_graphs = [G for G in all_G if G is not None]
    if not valid_graphs:
        print("No valid graphs found!")
        return []

    # 2. BFS Pathfinding
    print("Finding positive correlation paths...")
    all_positive_paths = []
    for G in tqdm(valid_graphs):



        

        
        paths = bfs_paths_icesheet15(G, max_length)
        all_positive_paths.append(paths)


        # we need to uncomment to get bfs_paths_seaice5_icesheet10


        # paths = bfs_paths_icesheet15(G, max_length)
        # all_positive_paths.append(paths)

        #print(paths)
    
    # 3. Monte Carlo Testing
    print("Running significance tests...")





    significant_paths = []
    for G, paths in zip(valid_graphs, all_positive_paths):


        
        
        if paths:
            (max_pos, p_pos), _ = parallel_monte_carlo(G, paths, sic_thresholds, ice_thresholds, M, n_workers)

            


            #print(max_pos,p_pos)



            sig_paths = [path for path in paths if is_significant_path(G, path, p_pos)]
            significant_paths.append(sig_paths)
            print('G',G)
            print('paths',sig_paths)
        else:
            significant_paths.append([])
    
    # 4. Visualization
    if any(sig_paths for sig_paths in significant_paths):
        #plot_positive_paths(valid_graphs, significant_paths, time_range)
        print('we ve significant paths')
    else:
        print("No significant paths found for visualization")
    
    return significant_paths


from collections import deque
#it will start from ice sheet nodes and end with ice sheet nodes

from collections import deque

def bfs_paths_icesheet15(G, max_length=15):
    """
    Find all simple positive-edge paths of EXACT length 15:
      - Start node is ice-sheet: L(...)
      - End node is ice-sheet: L(...)
      - All nodes are ice-sheet nodes
      - Total path length = 15

    Parameters
    ----------
    G : networkx.Graph or DiGraph
        Graph with edge attribute 'weight'.
    max_length : int
        Exact path length (default = 15)

    Returns
    -------
    list[list[str]]
        All valid ice-sheet-only paths.
    """

    is_ice_sheet = lambda n: isinstance(n, str) and n.startswith("L(")

    ice_sheet_nodes = [n for n in G.nodes if is_ice_sheet(n)]
    valid_paths = []

    for start in ice_sheet_nodes:
        queue = deque([(start, [start])])

        while queue:
            node, path = queue.popleft()
            k = len(path)

            # If exact length reached, record and stop expanding
            if k == max_length:
                valid_paths.append(path)
                continue

            # Do not exceed length
            if k > max_length:
                continue

            for nbr in G.neighbors(node):
                # No cycles
                if nbr in path:
                    continue

                # Positive-edge constraint
                if G[node][nbr].get("weight", 0) <= 0:
                    continue

                # Ice-sheet only constraint
                if not is_ice_sheet(nbr):
                    continue

                queue.append((nbr, path + [nbr]))

    return valid_paths

from collections import deque


## it will consist of first 5 sea ice nodes and last 10 ice sheet nodes

def bfs_paths_seaice5_icesheet10(G,max_length=15):
    """
    Find all simple positive-edge paths of EXACT length 15:
      - First 5 nodes are sea-ice nodes:  O(...)
      - Last 10 nodes are ice-sheet nodes: L(...)
    Total length = 5 + 10 = 15

    Parameters
    ----------
    G : networkx.Graph or DiGraph
        Graph with edge attribute 'weight'.
    Returns
    -------
    list[list[str]]
        All valid paths (each a list of node labels).
    """

    is_sea_ice   = lambda n: isinstance(n, str) and n.startswith("O(")
    is_ice_sheet = lambda n: isinstance(n, str) and n.startswith("L(")

    # We enforce exact length 15
    MAX_LEN = 15
    SEA_LEN = 5
    ICE_LEN = 10

    sea_ice_nodes = [n for n in G.nodes if is_sea_ice(n)]
    valid_paths = []

    for start in sea_ice_nodes:
        # start must be sea-ice
        queue = deque([(start, [start])])

        while queue:
            node, path = queue.popleft()
            k = len(path)

            # If we reached length 15, validate the whole pattern
            if k == MAX_LEN:
                if all(is_sea_ice(x) for x in path[:SEA_LEN]) and all(is_ice_sheet(x) for x in path[SEA_LEN:]):
                    valid_paths.append(path)
                continue

            # Decide what type the NEXT node must be
            # positions are 1-indexed in description, but here use k (current length)
            # If k < 5: we are still building sea-ice part => next must be sea-ice
            # If k >= 5: now in ice-sheet part => next must be ice-sheet
            require_next_sea = (k < SEA_LEN)

            for nbr in G.neighbors(node):
                # simple path constraint
                if nbr in path:
                    continue

                # positive edge constraint
                if G[node][nbr].get("weight", 0) <= 0:
                    continue

                # enforce node-type constraint for next node
                if require_next_sea:
                    if not is_sea_ice(nbr):
                        continue
                else:
                    if not is_ice_sheet(nbr):
                        continue

                queue.append((nbr, path + [nbr]))

    return valid_paths



from collections import deque


### It will start from sea ice nodes and end with three ice sheet nodes

def bfs_positive_paths(G, max_length):
    """
    Find all simple positive-edge paths that start on sea-ice nodes (O…)
    and end with THREE consecutive ice-sheet nodes (L…L…L…).

    Parameters
    ----------
    G : networkx.Graph
        Graph with 'weight' on edges.
    max_length : int
        Hard cap on the total number of nodes in any returned path.

    Returns
    -------
    list[list[str]]
        Paths that satisfy the criteria.
    """
    sea_ice_nodes   = [n for n in G if n.startswith('O(')]
    is_ice_sheet    = lambda n: n.startswith('L(')
    paths = []

    for start in sea_ice_nodes:
        queue   = deque([(start, [start])])
        visited = {start}

        while queue:
            node, path = queue.popleft()

            # ── Check end condition ──────────────────────────────────────────
            if (len(path) >= 4                                 # need room for O… + L,L,L
                and len(path) <= max_length
                and all(is_ice_sheet(n) for n in path[-3:])):  # last 3 are ice sheets
                paths.append(path)
                # do NOT continue exploring from here—comment out next line
                continue

            # ── Depth limit ─────────────────────────────────────────────────
            if len(path) >= max_length:
                continue

            # ── Positive-edge expansion ─────────────────────────────────────
            for nbr in G.neighbors(node):
                if nbr not in visited and G[node][nbr].get('weight', 0) > 0:
                    visited.add(nbr)
                    queue.append((nbr, path + [nbr]))

    return paths



# def bfs_positive_paths(G, max_length):
#     """BFS for paths between sea ice (O) and ice sheets (L) with positive edges"""
#     # Get all sea ice (O) and ice sheet (L) nodes
#     sea_ice_nodes = [n for n in G.nodes if n.startswith('O(')]
#     ice_sheet_nodes = [n for n in G.nodes if n.startswith('L(')]
#     ice_sheet_set = set(ice_sheet_nodes)
    
#     paths = []
#     for start in sea_ice_nodes:
#         queue = deque([(start, [start])])
#         visited = set([start])
        
#         while queue:
#             node, path = queue.popleft()
            
#             # Found path to ice sheet
#             if node in ice_sheet_set and len(path) > 1:
#                 paths.append(path)
#                 continue
                
#             if len(path) >= max_length:
#                 continue
                
#             # Explore positive edges only
#             for neighbor in G.neighbors(node):
#                 if neighbor not in visited and G[node][neighbor]['weight'] > 0:
#                     visited.add(neighbor)
#                     queue.append((neighbor, path + [neighbor]))
    
#     return paths

def is_significant_path(G, path, p_pos, alpha=0.05):
    """Check if path has at least one significant positive edge"""
    return any(p_pos < alpha 
              for u,v in zip(path[:-1], path[1:]) 
              if G.has_edge(u,v))

def parallel_monte_carlo(G, paths, sic_thresholds, ice_thresholds, M=500, n_workers=4):
    """Parallel Monte Carlo simulation for significance testing"""
    # Get original data from graph attributes
    ocean_points = G.graph['ocean_points']
    land_points = G.graph['land_points']
    sic_data = np.zeros((np.max(ocean_points[:,0])+1, np.max(ocean_points[:,1])+1))
    ice_data = np.zeros((np.max(land_points[:,0])+1, np.max(land_points[:,1])+1))
    
    # Reconstruct original data arrays from edges
    for u, v, data in G.edges(data=True):
        if u.startswith('O(') and v.startswith('L('):
            o_row, o_col = map(int, u[2:-1].split(','))
            l_row, l_col = map(int, v[2:-1].split(','))
            sic_data[o_row, o_col] = data['sic_data']
            ice_data[l_row, l_col] = data['ice_data']
    
    # Calculate observed scores
    observed_pos = []
    observed_neg = []
    for path in paths:
        max_pos = 0
        max_neg = 0
        for i in range(len(path)-1):
            u, v = path[i], path[i+1]
            if G.has_edge(u, v):
                weight = G[u][v]['weight']
                if weight > max_pos:
                    max_pos = weight
                elif weight < -max_neg:
                    max_neg = -weight
        observed_pos.append(max_pos)
        observed_neg.append(max_neg)
    
    max_observed_pos = max(observed_pos) if observed_pos else 0
    max_observed_neg = max(observed_neg) if observed_neg else 0
    
    # Parallel processing
    def process_chunk(chunk_size):
        null_pos = []
        null_neg = []
        for _ in range(chunk_size):
            # Shuffle while preserving ocean/land structure
            shuffled_ice = np.random.permutation(ice_data.reshape(-1)).reshape(ice_data.shape)
            G_shuffled = create_graph(sic_data, shuffled_ice, sic_thresholds, ice_thresholds)
            
            if G_shuffled:
                for path in paths:
                    max_p = 0
                    max_n = 0
                    for i in range(len(path)-1):
                        u, v = path[i], path[i+1]
                        if G_shuffled.has_edge(u, v):
                            weight = G_shuffled[u][v]['weight']
                            if weight > max_p:
                                max_p = weight
                            elif weight < -max_n:
                                max_n = -weight
                    null_pos.append(max_p)
                    null_neg.append(max_n)
        return null_pos, null_neg
    
    # Run simulations
    chunk_size = max(1, M // n_workers)
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        futures = [executor.submit(process_chunk, chunk_size) for _ in range(n_workers)]
        results = [f.result() for f in futures]
    
    # Combine results
    null_pos = np.concatenate([r[0] for r in results])
    null_neg = np.concatenate([r[1] for r in results])
    
    # Calculate p-values
    p_pos = (np.sum(null_pos >= max_observed_pos) + 1) / (len(null_pos) + 1)
    p_neg = (np.sum(null_neg >= max_observed_neg) + 1) / (len(null_neg) + 1)
    
    return (max_observed_pos, p_pos), (max_observed_neg, p_neg)

def plot_positive_paths(all_G, significant_paths, time_range):
    """Visualize significant positive paths"""
    plt.figure(figsize=(12, 10))
    
    # Combine all graphs for layout
    combined_G = nx.compose_all(all_G)
    pos = {node: (int(node.split(',')[0][2:]), int(node.split(',')[1][:-1])) 
           for node in combined_G.nodes()}
    
    # Draw all nodes
    nx.draw_networkx_nodes(combined_G, pos, node_size=3, node_color='gray', alpha=0.1)
    
    # Draw significant paths
    for G, paths in zip(all_G, significant_paths):
        for path in paths:
            edges = [(u,v) for u,v in zip(path[:-1], path[1:]) 
                    if G.has_edge(u,v)]
            nx.draw_networkx_edges(G, pos, edgelist=edges, 
                                 edge_color='red', width=2.5, alpha=0.7)
    
    # Highlight endpoints
    starts = {path[0] for paths in significant_paths for path in paths}
    ends = {path[-1] for paths in significant_paths for path in paths}
    
    nx.draw_networkx_nodes(combined_G, pos, nodelist=starts,
                         node_color='lime', node_size=80, alpha=0.8)
    nx.draw_networkx_nodes(combined_G, pos, nodelist=ends,
                         node_color='magenta', node_size=80, alpha=0.8)
    
    plt.title(f"Significant Positive Paths ({time_range[0]}-{time_range[1]})\n"
              f"Total: {sum(len(p) for p in significant_paths)} paths")
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"positive_paths_{time_range[0]}_{time_range[1]}.png", dpi=300)
    plt.close()

# Your existing parallel_monte_carlo can remain unchanged

def get_thresholds(data):
    """Calculate median, Q3, and upper bound (UB) thresholds"""
    flat_data = data[data != 0]  # Remove zeros

    if len(flat_data) == 0:
        return {'median': 0, 'q3': 0, 'ub': 0}

    median = np.median(flat_data)
    q1 = np.percentile(flat_data, 25)
    q3 = np.percentile(flat_data, 75)
    iqr = q3 - q1
    ub = q3 + 1.5 * iqr  # Upper bound

    return {
        'median': median,
        'q3': q3,
        'ub': ub
    }

# 6. Main Analysis Function
def analyze_ice_relationships(sic_data, ice_data, time_idx, sic_thresholds, ice_thresholds):

   
   
    G = create_graph(sic_data, ice_data, sic_thresholds, ice_thresholds)


    print('time_index',time_idx)

    print(f"Found {len(G.edges)} significant ocean-land connections")

    
    if G is not None:
        print(f"Found {len(G.edges)} significant ocean-land connections")
        #for edge in G.edges(data=True):
            # print(f"{edge[0]} ↔ {edge[1]} | "
            #     f"Weight: {edge[2]['weight']} | "
            #     f"Distance: {edge[2]['distance']:.1f} grid cells")
    else:
        print(f"Graph could not be created for time index {time_idx}")






    return G

        
        

# 7. Data Preprocessing
def preprocess_data(sic_data, ice_data, time_idx):
    """Prepare data for analysis"""
    sic_processed = np.where(sic_data[time_idx] < 0, -sic_data[time_idx], 0)
    ice_processed = np.where(ice_data[time_idx] < 0, -ice_data[time_idx], 0)
    return sic_processed, ice_processed


def count_median_to_q3(data, thresholds):
    median = thresholds['median']
    q3 = thresholds['q3']
    return ((data > median) & (data <= q3)).sum()








import numpy as np

 # Adjust as needed
all_sic = []
all_ice = []


sic_for_saving = []
ice_sheets_for_saving = []

for time_idx in range(0,120):
    # Preprocess data
    sic_processed, ice_processed = preprocess_data(sic_for_graph_building, ice_sheets_for_graph_building, time_idx)
    
    # Stack data for global threshold calculation

    sic_for_saving.append(sic_processed)
    ice_sheets_for_saving.append(ice_processed)

    all_sic.append(sic_processed.flatten())
    all_ice.append(ice_processed.flatten())

# Concatenate all data across time range
stacked_sic = np.concatenate(all_sic)
stacked_ice = np.concatenate(all_ice)

# Compute thresholds
sic_thresh = get_thresholds(stacked_sic)
ice_thresh = get_thresholds(stacked_ice)

print("\nGlobal Thresholds from Time 0 to 121:")
print(f"SIC: median={sic_thresh['median']:.2f}, q3={sic_thresh['q3']:.2f}, ub={sic_thresh['ub']:.2f}")
print(f"Ice: median={ice_thresh['median']:.2f}, q3={ice_thresh['q3']:.2f}, ub={ice_thresh['ub']:.2f}")


sic_for_saving = np.array(sic_for_saving)
ice_sheets_for_saving = np.array(ice_sheets_for_saving)

print(np.array(sic_for_saving).shape)
print(np.array(ice_sheets_for_saving).shape)


import pickle

# Replace with your actual data
# sic_for_graph_building = ...
# ice_sheets_for_graph_building = ...

# Save the objects to a pickle file
with open('sic_and_ice_diffrence_data.pkl', 'wb') as f:
    pickle.dump({
        'sic_for_graph_building': sic_for_saving,
        'ice_sheets_for_graph_building': ice_sheets_for_saving
    }, f)




In [ ]:
# Main script
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs





import matplotlib.pyplot as plt
import networkx as nx
import pickle
import random


import matplotlib.pyplot as plt
import networkx as nx
import pickle

def plot_paths_graph_unified_directed(all_G, all_paths, thres_def, time_range):
    """Construct unified directed graph from only the path edges and plot it."""

    final_G = nx.DiGraph()
    final_pos = {}

    for G, paths in zip(all_G, all_paths):
        if not paths:
            print("No paths found for this graph, skipping...")
            continue

        for path in paths:
            for i in range(len(path) - 1):
                u, v = path[i], path[i + 1]

                # Add nodes and edge (with data if available)
                if u not in final_G:
                    final_G.add_node(u, **G.nodes[u])
                if v not in final_G:
                    final_G.add_node(v, **G.nodes[v])

                edge_data = G.get_edge_data(u, v, default={})
                final_G.add_edge(u, v, **edge_data)

                # Position from coordinate in name (e.g., 'O(3,4)')
                for node in (u, v):
                    if node not in final_pos:
                        try:
                            y, x = map(int, node[2:-1].split(','))
                            final_pos[node] = (x, y)
                        except:
                            continue

    if not final_G:
        print("No graph to plot. Exiting.")
        return

    # Color nodes
        # Collect all start nodes from all paths
    start_nodes = set(path[0] for paths in all_paths for path in paths if path)

    # Color nodes
    node_colors = []
    for node in final_G.nodes:
        label = final_G.nodes[node].get('label', '')
        if label == 'O':
            if node in start_nodes:
                node_colors.append('darkgreen')  # Start node
            else:
                node_colors.append('lightgreen')  # Other sea ice
        elif label == 'L':
            node_colors.append('magenta')       # Ice sheet
        else:
            node_colors.append('gray')          # Unknown


    # Plot
    plt.figure(figsize=(14, 12))
    ax = plt.gca()
    nx.draw(
        final_G,
        pos=final_pos,
        node_color=node_colors,
        edge_color='black',
        node_size=100,
        alpha=0.8,
        with_labels=False,
        arrows=True,
        arrowsize=20,
        ax=ax
    )

    ax.invert_yaxis()
    ax.set_xlabel('X (Grid Column Index)')
    ax.set_ylabel('Y (Grid Row Index)')
    ax.set_title(f"Unified Directed Graph | Time Step {time_range} | {thres_def}", pad=20)

    # Save image
    filename = f"xy_paths_graph_UNIFIED_{thres_def}_{time_range}.png"
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    print(f"Saved unified graph plot to: {filename}")

    # Save graph to pickle
    with open(f"without_cycle_graph_end_with_3_ice_sheets_{thres_def}_{time_range}.pkl", 'wb') as f:
        pickle.dump({'graph': final_G, 'pos': final_pos}, f)
    print(f"Saved unified graph to: unified_graph_{time_range}.pkl")



#from path_plotting import plot_paths_graph_unified,plot_paths_graph_unified_directed


# Threshold definition


# Date indices mapping
date_indices = {
    "Nov": 0,      # Nov 1, 2019
    "Dec": 30,     # Dec 1, 2019
    "Jan": 61,     # Jan 1, 2020
    "Feb": 92      # Feb 1, 2020
}

# Month-wise difference index ranges (start, end)
month_ranges = {
    #"Nov": (0, 29),    # 30 days
    "Dec": (30, 60),   # 31 days
    #"Jan": (61, 91),   # 31 days
    #"Feb": (92, 120)   # 29 days (leap year)
}

chunk_size = 7  # Number of days per chunk

for month_name, (start_idx, end_idx) in month_ranges.items():
    print(f"\nProcessing {month_name}...")

    all_diff_indices = list(range(start_idx, end_idx + 1))
    total_diffs = len(all_diff_indices)

    for chunk_start in range(0, total_diffs, chunk_size):
        chunk_end = min(chunk_start + chunk_size - 1, total_diffs - 1)
        chunk_indices = all_diff_indices[chunk_start:chunk_end + 1]
        time_range = (chunk_indices[0], chunk_indices[-1])

        print(f"  - Chunk: Days {chunk_start + 1}-{chunk_end + 1} → Indices {time_range}")

        all_G = []
        for time_idx in range(time_range[0], time_range[1] + 1):
            # Preprocess data
            sic_processed, ice_processed = preprocess_data(
                sic_for_graph_building,
                ice_sheets_for_graph_building,
                time_idx
            )

            G = analyze_ice_relationships(
                sic_processed,
                ice_processed,
                time_idx,
                sic_thresh,
                ice_thresh
            )

            all_G.append(G)

        # # Find statistically significant positive paths
        sig_paths = find_positive_paths(
            all_G,
            sic_thresh,
            ice_thresh,
            time_range,
            max_length=11,
            M=500
        )


        print('now all the paths')

     

        # Custom label for file suffix
        file_suffix = f"{month_name}_days_{chunk_start + 1}-{chunk_end + 1}"

        # # Call plotting functions
        plot_paths_graph_unified_directed(all_G, sig_paths, thres_def, file_suffix)

 
            
        
        


  

        #plot_paths_xy_coordinates(all_G,sig_paths,thres_def,file_suffix)
